In [1]:
import xarray as xr
import xee
import ee

# Initialize Earth Engine
ee.Initialize()

# Define region of interest (example geometry)
geometry = ee.Geometry.Polygon([
    [
        [106.3495, -6.4493],
        [107.3300, -6.4493],
        [107.3300, -5.9005],
        [106.3495, -5.9005]
    ]
])

# Get SAR images from GEE
sar_before = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterDate('2019-12-20', '2019-12-29')
    .filterBounds(geometry)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .select('VV')
    .mean()
    .clip(geometry)
)

sar_after = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterDate('2019-12-30', '2020-01-03')
    .filterBounds(geometry)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .select('VV')
    .mean()
    .clip(geometry)
)

In [7]:
# Open as xarray Dataset using xee (this downloads the data)
ds_before = xr.open_dataset(
    ee.ImageCollection([sar_before]),
    engine='ee',
    scale=30,
    crs='EPSG:4326',
    geometry=geometry
)
ds_after = xr.open_dataset(
    ee.ImageCollection([sar_after]),
    engine='ee',
    scale=30,
    crs='EPSG:4326',
    geometry=geometry
)

ds_before # Check dataset structure

<xarray.Dataset> Size: 28B
Dimensions:  (time: 1, lon: 1, lat: 1)
Coordinates:
  * time     (time) int64 8B 0
  * lon      (lon) float64 8B 121.3
  * lat      (lat) float64 8B 8.55
Data variables:
    VV       (time, lon, lat) float32 4B ...
Attributes:
    crs:      EPSG:4326